# 3. Using `yadg` to process multiple files into one data archive
If one wishes to process multiple files, whether of the same or different filetypes, into a single data archive, `yadg` offers a convenient way of doing so by using the `yadg process` command coupled with a *DataSchema*. The usage of the `yadg process` command can be accessed using the `--help` argument:

In [ ]:
!yadg process --help

As shown above, the `infile` argument is a *DataSchema* file (JSON or YAML) specifying which files and folders are to be processed, and how. The *DataSchema* is documented in the [`dgbowl_schemas` package](https://dgbowl.github.io/dgbowl-schemas/master/apidoc/dgbowl_schemas.yadg.dataschema.html#module-dgbowl_schemas.yadg.dataschema); we will highlight the main functionalities below.

The `outfile` argument specifies the path of the output NetCDF file. Unlike in the [`yadg extract` usage](02_extract.ipynb), here the NetCDF file contains a `Datatree` as opposed to a `xarray.Dataset`.

## 3.1 Main features of a *DataSchema*
The default *DataSchema* for `yadg-5.1` is *DataSchema-5.1*, which is composed of the following elements:

  - `version`, which must be set to `"5.1"`;
  - `metadata`, an optional entry containing provenance information;
  - `step_defaults`, an optional entry providing default timezones, locales and file encoding for all steps,
  - `steps`, enumerating instructions for parsing of sets of files.

Note that previous versions of *DataSchemas* can be still used with `yadg-5.1`, as they will be automatically updated to the latest version, using default settings as necessary.

### 3.1.1 *DataSchema->`version`* and *DataSchema->`metadata`*

    version: "5.1"
    metadata:
        provenance: manual

The above example shows a sample `metadata` section of a valid *DataSchema-5.1*. The `metadata` entry is optional and can be an empty dict, i.e. `{}`.

### 3.1.2 *DataSchema->`step_defaults`*

    step_defaults:
        timezone: Europe/Berlin

The above example shows a sample `step_defaults` section of a valid *DataSchema-5.1*. The whole section is optional. However, **specifying at least the `step_defaults->timezone` is good practice**, as the `timezone` is used to make all `DateTime` objects within `yadg` timezone-aware. The specified `timezone` should be that of the data, not of the computer where the data is being processed. A wrong `timezone` might lead to wrong unix timestamps, but more crucially, to relative off-by-hour errors (e.g. when measurements span daylight savings adjustment) or even worse errors when combining data containing timezone-aware timestamps with other, unannotated data.

Other optional entries are the `locale`, which switches on locale-aware processing of certain text file types (decimal and thousand separators), and `encoding`, which adjusts the default encoding (`utf-8`) for all files.

### 3.1.3 *DataSchema->`steps`*
    
    steps:
      - extractor:
          filetype: "eclab.mpr"
        input:
            folders: ["process/data"]
            suffix: "mpr"
        tag: electro

The above example shows one *step*, i.e. a block in the `steps` enumeration. The following entries are required:

  - `extractor->filetype`, specifying the filetype of the matched files,
  - `input`, specifying files to be processed directly, or matched within the specified folders using filename components.

The `extractor` entry allows individual configuration of `timezone`, `encoding`, and `locale`, overriding the defaults from `step_defaults` for the particular step. The `tag` entry will become the name of the `DataTree` node (or HDF5 group in the NetCDF files); the default tag is the step index.


## 3.2 Usage example: electrochemistry and gas chromatography
A sample *DataSchema* is located in the [`process/ds.5.1.yml`](process/ds.5.1.yml) file. It contains the same `metadata` and `step_defaults` sections as above, but it has two steps:

- the first processing any files present in the `process/data/` folder, which have a `mpr` suffix and contain `LSV` in their filename, treating them as an `eclab.mpr` filetype,
- the second processing a single specified `process/data/CuDura05_03.zip` archive, treating it as a `fusion.zip` filetype.

We can process this *DataSchema*, generating a NetCDF file in `process/output.nc`, as follows:


In [ ]:
!yadg process process/ds.5.1.yml process/output.nc

Remember that `yadg process` stores `DataTree` objects in the NetCDF file instead of a simple `xarray.Dataset`. We can have a look at what is in the file using the `xarray-datatree` package, as follows:

In [ ]:
from datatree import open_datatree
dt = open_datatree("process/output.nc")
dt

The structure of the `DataTree` can be seen above, with the overall yadg provenance attributes stored in the root node that is otherwise empty. The data is stored in the two child nodes, called `LSV` and `GC`, as specified using the `step->[x]->tag` entry. We can also access the individual leaves of the `DataTree` containing our data using dictionary notation:

In [ ]:
dt["LSV"]

As you can see, the first leaf, `LSV`, corresponds to the first *step* and contains 235 points along the `uts` dimension. In its attributes, the `original_metadata` extracted from the file is also stored. The data in the second leaf can be accessed analogously, using `dt["GC"]`.

Note that as the `uts` timestamps are referencing the Unix epoch, the data is guaranteed to be properly aligned in time, assuming the `timezone` of the data files has been set properly.

[Back to index](index.ipynb)